### Initialize, create dataframes

In [217]:
import pandas as pd
import numpy as np
import pickle

from datetime import datetime

df_elpriser = pd.read_csv('./dataset/elpriser_200601_202302.csv', delimiter=';', header=0)
df_bransle = pd.read_csv('./dataset/Drivmedel_200601_202302.csv', delimiter=';', header=0)
df_nyreg = pd.read_csv('./dataset/nyreg_200601_202302.csv', delimiter=';', header=0)

# Dictionary for translation swedish dates to english
swedish_to_english = {
    'jan': 'Jan',
    'feb': 'Feb',
    'mar': 'Mar',
    'apr': 'Apr',
    'maj': 'May',
    'jun': 'Jun',
    'jul': 'Jul',
    'aug': 'Aug',
    'sep': 'Sep',
    'okt': 'Oct',
    'nov': 'Nov',
    'dec': 'Dec'
}


### Elpriser

In [218]:
df_elpriser = df_elpriser.transpose()
df_elpriser = df_elpriser.reset_index()


df_elpriser.columns = ['Month','Elpris']

# Replace swedish naming with english in order for strptime to work
df_elpriser['Month'] = df_elpriser['Month'].replace(swedish_to_english, regex=True)

# Convert to proper date format
df_elpriser['Month'] = df_elpriser['Month'].apply(lambda x: datetime.strptime(x, "%y-%b").strftime("%Y-%m-%d"))
df_elpriser['Month'] = pd.to_datetime(df_elpriser['Month'])


# Replace commas with dots
df_elpriser['Elpris'] = df_elpriser['Elpris'].str.replace(',', '.')

# Object -> float
df_elpriser['Elpris'] = df_elpriser['Elpris'].astype(float)

# Check for NaN:s
print(df_elpriser[df_elpriser.isna().any(axis=1)])





Empty DataFrame
Columns: [Month, Elpris]
Index: []


### Bränsle

In [219]:
df_bransle = df_bransle.rename(columns={'Unnamed: 0':'Month'}) 

df_bransle['Month'] = pd.to_datetime(df_bransle['Month'])

# Rename to avoid name conflict with column in df "nyreg" 
df_bransle = df_bransle.rename(columns={'Diesel':'Diesel_fuel'})

# Replace commas with dots
df_bransle['E85'] = df_bransle['E85'].str.replace(',', '.')
df_bransle['Diesel_fuel'] = df_bransle['Diesel_fuel'].str.replace(',', '.')
df_bransle['BF95'] = df_bransle['BF95'].str.replace(',', '.')

# Object -> float
df_bransle[['E85', 'Diesel_fuel', 'BF95']] = df_bransle[['E85', 'Diesel_fuel', 'BF95']].astype(float)

# Check for NaN:s
print(df_elpriser[df_elpriser.isna().any(axis=1)])




Empty DataFrame
Columns: [Month, Elpris]
Index: []


### Nyregistrerade bilar

In [215]:
df_nyreg = df_nyreg.transpose()
df_nyreg = df_nyreg.reset_index()

# Adjust index so that first rownumber is 0
df_nyreg.index = df_nyreg.index - 1

df_nyreg.columns = ['Month','Bensin','Diesel','El','Elhybrid','Laddhybrid','Etanol','Gas']

# Drop reduntant label row now that columns are properly named
df_nyreg = df_nyreg.drop(df_nyreg.index[0]) 

# Alter date format to match the other dataframes
df_nyreg['Month'] = df_nyreg['Month'].str.replace('M', '-') + '-01'

df_nyreg['Month'] = pd.to_datetime(df_nyreg['Month']) 

# Insert columns holding the relative proportion of eaach fuel type.
for column in df_nyreg.columns:
    if column != 'Month':
        df_nyreg.insert(df_nyreg.columns.get_loc(column) + 1, column + '_andel', df_nyreg[column] / df_nyreg[['Bensin','Diesel','El','Elhybrid','Laddhybrid','Etanol','Gas']].sum(axis=1))

# Object -> float
for column in df_nyreg.columns:
    if column != 'Month':
        df_nyreg[[column]] = df_nyreg[[column]].astype(float)


# Check for NaN:s
print(df_nyreg[df_nyreg.isna().any(axis=1)])


Empty DataFrame
Columns: [Month, Elpris]
Index: []


### Sammanslagning

In [216]:
complete_df = pd.merge(df_bransle, df_elpriser, on='Month', how='inner').merge(df_nyreg, on='Month', how='inner') 

print(complete_df)

pickle.dump(complete_df, open('complete_df.pkl', 'wb'))




         Month    E85  Diesel_fuel   BF95  Elpris   Bensin  Bensin_andel   
0   2023-02-01  16.40        22.95  19.83   95.27   4256.0      0.223529  \
1   2023-01-01  15.58        24.17  19.84  108.88   3602.0      0.235702   
2   2022-12-01  14.71        23.64  18.89  269.37   4370.0      0.121798   
3   2022-11-01  15.73        25.56  20.42  162.11   4238.0      0.161171   
4   2022-10-01  16.34        27.60  21.57   87.58   4523.0      0.194789   
5   2022-09-01  17.05        24.21  19.06  191.98   5016.0      0.217642   
6   2022-08-01  17.29        24.44  19.76  178.04   5975.0      0.277997   
7   2022-07-01  19.56        25.25  21.61   83.23   5672.0      0.301879   
8   2022-06-01  20.57        26.37  23.54  122.19   6490.0      0.237086   
9   2022-05-01  18.70        23.73  21.52  106.19   7786.0      0.280718   
10  2022-04-01  18.63        24.86  21.01   93.22   5968.0      0.258915   
11  2022-03-01  18.95        25.18  21.31  103.40   6737.0      0.225370   
12  2022-02-